In [1]:
import scipy as sp
from openfermionpyscf import run_pyscf
from openfermion import *
from pyscf import mp, cc, fci
import numpy as np
import json

# 1. Data input and parameter setting

In [2]:
# read the molecule data
Molecules_Data = json.load(open('Molecules_Data.json', 'r'))
Molecules_Data

{'AlCl3': {'Internal': [['0.0000', '0.0000', '0.0000'],
   ['0.0000', '2.0875', '0.0000'],
   ['1.8078', '-1.0437', '0.0000'],
   ['-1.8078', '-1.0437', '0.0000']],
  'Molecules': ['Al1', 'Cl2', 'Cl3', 'Cl4'],
  'Name': 'AlCl3',
  'Principal': [['0.0000', '0.0000', '0.0000'],
   ['1.5089', '-1.4425', '0.0000'],
   ['0.4948', '2.0280', '0.0000'],
   ['-2.0037', '-0.5855', '0.0000']]},
 'AlF3': {'Internal': [['0.0000', '0.0000', '0.0000'],
   ['0.0000', '1.6390', '0.0000'],
   ['1.4194', '-0.8195', '0.0000'],
   ['-1.4194', '-0.8195', '0.0000']],
  'Molecules': ['Al1', 'F2', 'F3', 'F4'],
  'Name': 'AlF3',
  'Principal': [['0.0000', '0.0000', '0.0000'],
   ['1.5142', '0.6272', '0.0000'],
   ['-1.3003', '0.9978', '0.0000'],
   ['-0.2139', '-1.6250', '0.0000']]},
 'AlH3': {'Internal': [['0.0000', '0.0000', '0.0000'],
   ['0.0000', '1.5890', '0.0000'],
   ['1.3761', '-0.7945', '0.0000'],
   ['-1.3761', '-0.7945', '0.0000']],
  'Molecules': ['Al1', 'H2', 'H3', 'H4'],
  'Name': 'AlH3',
  'Prin

In [3]:
mole = 'H2'   # set the molucular
geometry = list(zip(map(lambda x:x[:-1], Molecules_Data['H2']['Molecules']), [tuple(map(float, x)) for x in Molecules_Data['H2']['Internal']]))
active_space_start = 0 
active_space_stop = 2

In [4]:
#set the parameters for pyscf
basis = 'sto-6g'# gaussian wavefuctions
multiplicity = 1  # multiplet
charge = 0 # molecular charge
description = 'ccsd' # mark for saving file

# set the calculator, 0-shut down, 1-turn on
run_scf = 1 
run_mp2 = 1
run_ccsd = 1
run_cisd = 0
run_fci = 1
delete_input = True
delete_output = True

# 2. Compute the hamiltonian of the system

In [5]:
# calculation from parameters
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_pyscf(molecule, run_scf=run_scf,run_mp2=run_mp2, run_cisd=run_cisd, run_ccsd=run_ccsd, run_fci=run_fci)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
print('MP2 energy of {} Hartree.'.format(molecule.mp2_energy))
print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
print('Nuclear repulsion energy between protons is {} Hartree.'.format(
    molecule.nuclear_repulsion))
for orbital in range(molecule.n_orbitals):
    print('Spatial orbital {} has energy of {} Hartree.'.format(
        orbital, molecule.orbital_energies[orbital]))
MolecularData.save(molecule)
molecule.load()

Hartree-Fock energy of -1.1252095979375198 Hartree.
MP2 energy of -1.1384431580364507 Hartree.
FCI energy of -1.145900469958307 Hartree.
Nuclear repulsion energy between protons is 0.7124087384491115 Hartree.
Spatial orbital 0 has energy of -0.5817274220336034 Hartree.
Spatial orbital 1 has energy of 0.6650468891904611 Hartree.


In [6]:
# get the hamiltonian for the molecular
molecule_hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices=range(
    active_space_start), active_indices=range(active_space_start, active_space_stop))
# change the hamiltonian to fermion operator
fermion_hamiltonian = get_fermion_operator(molecule_hamiltonian)

# J-W transformation for modified fermion operator
qubit_hamiltonian_JW = jordan_wigner(fermion_hamiltonian)
qubit_hamiltonian_JW.compress()
print('The Jordan-Wigner Hamiltonian in canonical basis follows:\n{}', format(qubit_hamiltonian_JW))

# B-K transformation for modified fermion operator
qubit_hamiltonian_BK = bravyi_kitaev(fermion_hamiltonian)
print('qubit_hamiltonian BK transfer fermion_hamiltonian', format(qubit_hamiltonian_BK))

The Jordan-Wigner Hamiltonian in canonical basis follows:
{} -0.10828642633408778 [] +
-0.045413740730899495 [X0 X1 Y2 Y3] +
0.045413740730899495 [X0 Y1 Y2 X3] +
0.045413740730899495 [Y0 X1 X2 Y3] +
-0.045413740730899495 [Y0 Y1 X2 X3] +
0.17287043695155 [Z0] +
0.1685408730798563 [Z0 Z1] +
0.12056020320710414 [Z0 Z2] +
0.16597394393800363 [Z0 Z3] +
0.1728704369515501 [Z1] +
0.16597394393800363 [Z1 Z2] +
0.12056020320710414 [Z1 Z3] +
-0.2206655790398497 [Z2] +
0.17467628158972687 [Z2 Z3] +
-0.2206655790398497 [Z3]
qubit_hamiltonian BK transfer fermion_hamiltonian (-0.10828642633408778+0j) [] +
(0.045413740730899495+0j) [X0 Z1 X2] +
(0.045413740730899495+0j) [X0 Z1 X2 Z3] +
(0.045413740730899495+0j) [Y0 Z1 Y2] +
(0.045413740730899495+0j) [Y0 Z1 Y2 Z3] +
(0.17287043695155+0j) [Z0] +
(0.1728704369515501+0j) [Z0 Z1] +
(0.16597394393800363+0j) [Z0 Z1 Z2] +
(0.16597394393800363+0j) [Z0 Z1 Z2 Z3] +
(0.12056020320710414+0j) [Z0 Z2] +
(0.12056020320710414+0j) [Z0 Z2 Z3] +
(0.1685408730798563+0j) 

## In this part we compute the ground state of the molecula in normal way.
## It is used for the comparation with FQE method.

In [7]:
# change the qubit_hamiltonian_JW to sparse_operator 
sparse_operator = get_sparse_operator(qubit_hamiltonian_JW)
H_csr = sparse_operator.tocsr()

# calculater the eigenvalue and eigen state of sparse_operator
V, D = sp.sparse.linalg.eigs(sparse_operator)

# compute the energy of the ground state
E_theory = np.min(V)
print('E_theory',E_theory)
Ha = sparse_operator.todense()
Ha = np.mat(Ha)

E_theory (-1.1459004699583069+1.9643867502022242e-17j)


In [8]:
# get the initial ansaz state
H0 = np.diagonal(Ha)
jj = np.argmin(H0)
initial_state = np.zeros(len(Ha))
initial_state[jj] = 1       # initial ansaz state

# 3. Simulation with FullQuantumEigensolver(FQE)

In [9]:
from projectq import MainEngine
from projectq.ops import All, CNOT, H, Measure, X, Z, C, Y, CX, QubitOperator, StatePreparation, FlipBits
from projectq.backends import CircuitDrawer
import math
import numpy as np

In [10]:
class FullQuantumEigensolver:
    def __init__(self, Gatelist, active_space_stop, active_space_start, working_state, gamma=1):
        self.gamma = gamma                                                   # parameter gamma
        self.ancilla_num = math.ceil(math.log(len(Gatelist)) / math.log(2))  # number of  ancilla qubits
        self.working_num = (active_space_stop - active_space_start) * 2      # number of working qubits
        self.gates_list = list(map(lambda x:x[0], Gatelist))                 # the gates set for fqe (pauli product items of hamiltonian)
        self.ancilla_state = self.cal_coe_list(Gatelist)                     # the initial state of ancilla qubits
        self.working_state = working_state                                   # the inital state of working qubits
        self.flip_position = []                                              # the flip postion for graycode                                          
        for i in range(self.ancilla_num):
            self.flip_position = self.flip_position + [i] + self.flip_position

    # run FQE, iteration times - n
    def start(self, n):
        for i in range(n):
            self.working_state = self.reverseMap(self.working_state)
            eng = MainEngine()
            final_state = self.quantumIter(eng)
            final_state = np.array(list(map(lambda x:x[1], final_state.items())))
            possibility = sum(abs(final_state)**2)
            final_state = final_state / math.sqrt(possibility)
            Energy = self.cal_E(final_state, H_csr)
            self.log(i+1, final_state, possibility, Energy)
            self.working_state = final_state
    
    # calculate the initial state of ancilla qubits
    def cal_coe_list(self, Gatelist):
        coe_list = [0] + list(map(lambda x:x[1], Gatelist))
        coe_list.extend([0 for x in range((1 << self.ancilla_num) - len(coe_list))])
        coe_list = np.array(coe_list)
        coe_list = -self.gamma * coe_list
        coe_list[0] = 1
        coe_list = coe_list / math.sqrt(sum(coe_list * coe_list))
        gray_code = [0]
        for i in range(self.ancilla_num):
            gray_code += [(1 << i) + gray_code[-j-1] for j in range(1 << i)]
        coe_list_m = np.zeros(len(coe_list))
        for i in range(len(coe_list)):
            coe_list_m[gray_code[i]] = coe_list[i]
    
        return coe_list_m
    
    # with hamiltonian-H_csr, compute the energy
    def cal_E(self, state, H_csr):
        state = np.matrix(state)
        state = sp.sparse.csr_matrix(state)
        E = np.real((state.conjugate().tocsr() * H_csr * state.transpose()).toarray()[0][0])
        return E
    
    # iteration
    def quantumIter(self, eng):
        eng, qreg = self.circuit_build(eng)
        try:
            eng.flush()
        except:
            pass
        anc_state = '0' * self.ancilla_num

        return self.post_select(eng, qreg, anc_state)
    
    # reverse the quantum state, because the  projectq
    def reverseMap(self, state):
        length = len(state)
        rmap = np.zeros(length, dtype=np.int32)
        for i in range(length):
            rmap[i] = int("{:0>{}b}".format(i, self.working_num)[::-1], 2)
        state_new = np.zeros(len(state))
        for i, coe in enumerate(state):
            state_new[rmap[i]] = coe
            
        return state_new
        
    # bulid the quantum circuit
    def circuit_build(self, eng):
        # allocate qubit register
        ancillas = eng.allocate_qureg(self.ancilla_num)
        working_qubits = eng.allocate_qureg(self.working_num)
        
        # initial state
        StatePreparation(self.ancilla_state) | ancillas
        StatePreparation(self.working_state) | working_qubits
        FlipBits(-1) | ancillas
        
        # quantum gate
        for i, gates in enumerate(self.gates_list):
            X | ancillas[self.flip_position[i]]
            for gate in gates:
                C(eval(gate[1]), self.ancilla_num) | (ancillas, working_qubits[gate[0]])
        i += 1
        while i < len(self.flip_position):

            X | ancillas[self.flip_position[i]]
            i += 1
        
        X | ancillas[self.ancilla_num-1]
        FlipBits(-1) | ancillas

        for i in range(self.ancilla_num):
            H | ancillas[i]
            
        return eng, ancillas+working_qubits
    
    # post select the quantum state
    def post_select(self, eng, qreg, anc_state):
        dct = {}
        for i in range(len(self.working_state)):
            num2 = bin(i)[2:]
            num2 = '0' * (self.working_num - len(num2)) + num2
            dct[num2] = eng.backend.get_amplitude(anc_state+num2, qreg)
        return dct
    
    # log 
    def log(self, i, final_state, possibility, energy):
        print('The {}\'s iteration'.format(i))
        print("Energy: {}".format(energy))
        print('The final remain state after select:')
        print(final_state)
        print('select possibility: {}'.format(possibility))
        
    # print latex code for the circuit
    def plot_circuits(self):
        drawing_engine = CircuitDrawer()
        eng = MainEngine(drawing_engine)
        eng = self.circuit_build(eng)[0]
        eng.flush()
        print(drawing_engine.get_latex())
        

In [11]:
# get the gatelist (pauli product items of the hamiltonian after JW tranformation
Gatelist = list(qubit_hamiltonian_JW.terms.items())    

In [12]:
# get the FullQuantumEigensolver
FQE = FullQuantumEigensolver(Gatelist, active_space_stop, active_space_start, initial_state)

In [13]:
# estimate the iteration times - K
epsilon = 1e-4
gamma = 1
H1 = sorted(H0)
K = math.log(epsilon / len(H0)) / math.log((1 - gamma * H1[1]) / (1 - gamma * H1[0]))
K = math.ceil(K)

In [14]:
# run FQE (K iterations)
FQE.start(K)

In [15]:
# print the latex code for circuit
FQE.plot_circuits()

\documentclass{standalone}
\usepackage[margin=1in]{geometry}
\usepackage[hang,small,bf]{caption}
\usepackage{tikz}
\usepackage{braket}
\usetikzlibrary{backgrounds,shadows.blur,fit,decorations.pathreplacing,shapes}

\begin{document}
\begin{tikzpicture}[scale=0.8, transform shape]

\tikzstyle{basicshadow}=[blur shadow={shadow blur steps=8, shadow xshift=0.7pt, shadow yshift=-0.7pt, shadow scale=1.02}]\tikzstyle{basic}=[draw,fill=white,basicshadow]
\tikzstyle{operator}=[basic,minimum size=1.5em]
\tikzstyle{phase}=[fill=black,shape=circle,minimum size=0.1cm,inner sep=0pt,outer sep=0pt,draw=black]
\tikzstyle{none}=[inner sep=0pt,outer sep=-.5pt,minimum height=0.5cm+1pt]
\tikzstyle{measure}=[operator,inner sep=0pt,minimum height=0.5cm, minimum width=0.75cm]
\tikzstyle{xstyle}=[circle,basic,minimum height=0.35cm,minimum width=0.35cm,inner sep=-1pt,very thin]
\tikzset{
shadowed/.style={preaction={transform canvas={shift={(0.5pt,-0.5pt)}}, draw=gray, opacity=0.4}},
}
\tikzstyle{swapstyle}=[inne